In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import zipfile
import os
import pandas as pd

extraction_dir = '/kaggle/input/movie-classifier/Multi_Label_dataset'

csv_path = os.path.join(extraction_dir, 'train.csv')

df = pd.read_csv(csv_path)

df

,Id,Genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,N/A,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
0,tt0086425,"['Comedy', 'Drama']",0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,tt0085549,"['Drama', 'Romance', 'Music']",0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,tt0086465,['Comedy'],0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0086567,"['Sci-Fi', 'Thriller']",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,tt0086034,"['Action', 'Adventure', 'Thriller']",1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,tt2409818,"['Action', 'Crime', 'Thriller']",1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7250,tt2062622,"['Animation', 'Comedy', 'Family']",0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7251,tt2442502,['Comedy'],0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7252,tt3455850,['Documentary'],0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
labels = df.drop(['Id', 'Genre'], axis = 1).columns
labels

Index(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror',
       'Music', 'Musical', 'Mystery', 'N/A', 'News', 'Reality-TV', 'Romance',
       'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western'],
      dtype='object')

In [4]:
image_names = df['Id']
image_names

multi_hot_encoding = df.drop(['Genre'], axis=1).set_index('Id').T.to_dict(orient = 'list')


/tmp/ipykernel_34/1711675911.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  multi_hot_encoding = df.drop(['Genre'], axis=1).set_index('Id').T.to_dict(orient = 'list')


In [5]:
import cv2
import numpy as np

images_dict = {}
for image_name in image_names:
    image_path = os.path.join(extraction_dir,'Images', image_name + '.jpg')
    img = cv2.imread(image_path)
    img = cv2.resize(img, (475, 300))
    images_dict[image_name] = img

In [6]:
#multi_hot_encoding

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Input

images = np.array([images_dict[image_name] for image_name in image_names[:500]])
labels = np.array([multi_hot_encoding[image_name] for image_name in image_names[:500]])

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_test = X_test / 255.0

model = Sequential([
    Input(shape=(300, 475, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(labels[0]), activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.optimizer.learning_rate.assign(0.01)



2024-07-03 12:50:30.276365: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 12:50:30.276514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 12:50:30.409887: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
print(len(labels[0]))
model.summary()

25


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 298, 473, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 149, 236, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 147, 234, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 73, 117, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 71, 115, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 35, 57, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 255360)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    32,686,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 25)             │         3,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,782,681 (125.06 MB)

 Trainable params: 32,782,681 (125.06 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.5384 - loss: 0.0570 - val_accuracy: 0.2200 - val_loss: 0.4438
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.5147 - loss: 0.0439 - val_accuracy: 0.3100 - val_loss: 0.4742
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.5832 - loss: 0.0311 - val_accuracy: 0.2900 - val_loss: 0.5583
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.5402 - loss: 0.0271 - val_accuracy: 0.2600 - val_loss: 0.5497
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.5338 - loss: 0.0146 - val_accuracy: 0.2400 - val_loss: 0.5971
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.5834 - loss: 0.0124 - val_accuracy: 0.2900 - val_loss: 0.6326
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.5536 - loss: 0.0104 - val_accuracy: 0.2200 - val_loss: 0.6471
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.5574 - loss: 0.0097 - val_accuracy: 0

In [18]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

predictions = model.predict(X_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2582 - loss: 1.2919
Test Loss: 1.2799662351608276
Test Accuracy: 0.25999999046325684
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


In [19]:
predictions[0]

array([2.0665017e-01, 5.1158602e-18, 2.7822099e-25, 2.7294067e-10,
       5.3352898e-01, 2.0845131e-07, 1.5825819e-21, 9.9999988e-01,
       1.4320765e-18, 9.7682369e-09, 9.1718769e-20, 3.4436607e-28,
       1.8959652e-16, 1.1417498e-20, 1.4681893e-14, 5.9281444e-33,
       0.0000000e+00, 1.9718764e-30, 3.3266164e-02, 1.3343830e-12,
       2.6863871e-30, 1.5993188e-15, 3.1270712e-11, 3.3468596e-06,
       9.8793925e-15], dtype=float32)

In [20]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Model
import tensorflow as tf

ResNet50_model = vgg16(weights='imagenet', input_shape=(300, 475, 3), include_top=False)

x = ResNet50_model.output
x = tf.keras.layers.Flatten()(x)
predictions = tf.keras.layers.Dense(25, activation='sigmoid')(x)
TL_model = Model(inputs = ResNet50_model.input, outputs = predictions)

TL_model.compile(optimizer='adam',loss = 'binary_crossentropy', metrics=['accuracy'])


In [15]:
for layer in ResNet50_model.layers:
    layer.trainable = False

In [21]:
TL_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100


2024-07-03 12:58:30.934625: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[32,1024,19,30]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,2048,10,15]{3,2,1,0}, f32[2048,1024,1,1]{3,2,1,0}), window={size=1x1 stride=2x2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-07-03 12:58:31.365220: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.430682488s
Trying algorithm eng0{} for conv (f32[32,1024,19,30]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,2048,10,15]{3,2,1,0}, f32[2048,1024,1,1]{3,2,1,0}), window={size=1x1 stride=2x2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-07-03 12:58:32.396019: E exter

13/13 ━━━━━━━━━━━━━━━━━━━━ 121s 4s/step - accuracy: 0.2324 - loss: 2.9842 - val_accuracy: 0.3200 - val_loss: 5810105.5000
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 419ms/step - accuracy: 0.3565 - loss: 0.4420 - val_accuracy: 0.3200 - val_loss: 3398572.7500
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 419ms/step - accuracy: 0.3584 - loss: 0.2783 - val_accuracy: 0.3200 - val_loss: 509213.0938
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 418ms/step - accuracy: 0.4479 - loss: 0.1903 - val_accuracy: 0.3200 - val_loss: 2435.5813
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 419ms/step - accuracy: 0.5032 - loss: 0.1274 - val_accuracy: 0.3200 - val_loss: 19.8924
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 416ms/step - accuracy: 0.5270 - loss: 0.0665 - val_accuracy: 0.3200 - val_loss: 10.2926
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 414ms/step - accuracy: 0.5697 - loss: 0.0282 - val_accuracy: 0.3200 - val_loss: 1.8245
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 418ms/step - accuracy: 0.5310 - loss: 0.0128 -

In [ ]:
loss, accuracy = TL_model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

predictions_TL = TL_model.predict(X_test)

In [22]:
fully_connected = Sequential([
    Dense(512, activation='relu'),
    Dense(256, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(len(labels[0]), activation='sigmoid') 
])(x)

dense_model = Model(inputs = ResNet50_model.input, outputs = fully_connected)

dense_model.compile(optimizer='adam',loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
dense_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 91s 3s/step - accuracy: 0.1807 - loss: 0.4741 - val_accuracy: 0.1200 - val_loss: 109.2773
Epoch 2/100


W0000 00:00:1720012583.349701     118 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 438ms/step - accuracy: 0.3820 - loss: 0.2183 - val_accuracy: 0.2400 - val_loss: 17.7470
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 438ms/step - accuracy: 0.5405 - loss: 0.1362 - val_accuracy: 0.1900 - val_loss: 2.4433
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 438ms/step - accuracy: 0.5298 - loss: 0.0488 - val_accuracy: 0.2900 - val_loss: 64.2491
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 436ms/step - accuracy: 0.5093 - loss: 0.0313 - val_accuracy: 0.2300 - val_loss: 106.3283
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 435ms/step - accuracy: 0.5147 - loss: 0.0265 - val_accuracy: 0.3000 - val_loss: 10.1482
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 437ms/step - accuracy: 0.4761 - loss: 0.0157 - val_accuracy: 0.3000 - val_loss: 284.2320
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 436ms/step - accuracy: 0.5165 - loss: 0.0190 - val_accuracy: 0.2900 - val_loss: 38.1245
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 437ms/step - accuracy: 0.4192 - loss: 0.0199 - val_accurac

In [ ]:
loss, accuracy = dense_model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

predictions_dense = dense_model.predict(X_test)

In [ ]:
def GenrePrediction(predictions):
    if len(predictions) != len(labels):
        raise ValueError("The length of predictions and genre_labels must be the same.")

    predictions = np.array(predictions)

    top_indices = np.argsort(predictions)[-3:][::-1]

    top_values = predictions[top_indices]

    diff1 = top_values[0] - top_values[1]
    diff2 = top_values[1] - top_values[2]

    classification = []

    threshold = np.std(predictions)
    
    if diff1 > threshold and diff2 > threshold:
        classification.append(labels[top_indices[0]])
    elif diff1 <= threshold and diff2 > threshold:
        classification.append(labels[top_indices[0]])
        classification.append(labels[top_indices[1]])
    else:
        classification.append(labels[top_indices[0]])
        classification.append(labels[top_indices[1]])
        classification.append(labels[top_indices[2]])

    return classification

